In [1]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm



import numpy as np
import os
import shutil

import tensorflow as tf

from tensorflow import keras

# from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers, initializers, regularizers, metrics
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras import layers

from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img


In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11228598565850897656
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4986830848
locality {
  bus_id: 1
  links {
  }
}
incarnation: 15378893915318036369
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1660 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


In [3]:
from io import BytesIO
import time, random
from tqdm import tqdm

from PIL import Image

In [4]:
BASE_DIR = 'D:/original_image/'
names = ["63building", "castle","general","indep_door","judgement_castle","lotte_tower","namsan","tapgol_park","seoulstation",\
        "coex","moonlight"]

tf.random.set_seed(1)

In [5]:
if not os.path.isdir(BASE_DIR + 'train/'):
    for name in names:
        os.makedirs(BASE_DIR + 'train/' + name)
        os.makedirs(BASE_DIR + 'val/' + name)
        os.makedirs(BASE_DIR + 'test/' + name)
        

In [6]:
orig_folders = ["/63building/", "/castle/","/general/", "/indep_door/","/judgement_castle/", "/lotte_tower/","/namsan/", "/tapgol_park/",\
               "/seoulstation/","/coex/","/moonlight/"]

            

In [7]:
for folder_idx, folder in enumerate(orig_folders):
    files = os.listdir(BASE_DIR + folder)
    number_of_images = len([name for name in files])
    n_train = int((number_of_images * 0.6) + 0.5)
    n_valid = int((number_of_images* 0.25) + 0.5)
    n_test = number_of_images - n_train - n_valid
    print(number_of_images, n_train, n_valid, n_test)
    for idx, file in enumerate(files):
        file_name = BASE_DIR + folder + file
        if idx < n_train:
            shutil.move(file_name, BASE_DIR + "train/" + names[folder_idx])
        elif idx < n_train + n_valid:
            shutil.move(file_name, BASE_DIR + "val/" + names[folder_idx])
        else:
            shutil.move(file_name, BASE_DIR + "test/" + names[folder_idx])

0 0 0 0
0 0 0 0
0 0 0 0
0 0 0 0
0 0 0 0
0 0 0 0
0 0 0 0
0 0 0 0
0 0 0 0
0 0 0 0
0 0 0 0


In [8]:
train_gen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
valid_gen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_gen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

#train에 저장
train_batches = train_gen.flow_from_directory(
    'D:/original_image/train',
    target_size = (240, 240),
    class_mode = 'sparse',
    batch_size = 2,
    shuffle=True,
    color_mode="rgb",
    classes=names
)

#val에 저장
val_batches = valid_gen.flow_from_directory(
    'D:/original_image/val',
    target_size = (240, 240),
    class_mode = 'sparse',
    batch_size = 2,
    shuffle=False,
    color_mode="rgb",
    classes=names
)

#테스트에 저장
test_batches = valid_gen.flow_from_directory(
    'D:/original_image/test',
    target_size = (240, 240),
    class_mode = 'sparse',
    batch_size = 2,
    shuffle=False,
    color_mode="rgb",
    classes=names
)

Found 6600 images belonging to 11 classes.
Found 2750 images belonging to 11 classes.
Found 1650 images belonging to 11 classes.


In [9]:
model = keras.models.Sequential()
model.add(layers.Conv2D(32, (3,3), strides=(1,1), padding="valid", activation='relu', input_shape=(240,240,3)))
model.add(layers.MaxPool2D(2,2))

In [10]:
train_batch = train_batches[0]
print(train_batch[0].shape)
print(train_batch[1])

test_batch = test_batches[0]
print(test_batch[0].shape)
print(test_batch[1])

(2, 240, 240, 3)
[4. 7.]
(2, 240, 240, 3)
[0. 0.]


In [11]:
# vgg 모델
vgg_model = tf.keras.applications.vgg16.VGG16()
print(type(vgg_model))
vgg_model.summary()

<class 'tensorflow.python.keras.engine.training.Model'>
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block

In [12]:
model = tf.keras.models.Sequential()
for layer in vgg_model.layers[0:-1]:
    model.add(layer)

In [13]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [14]:
# 마지막 레이어만 train해주면됨
#앞에 레이어 non trainable로 바꿔줌
for layer in model.layers:
    layer.trainable = False
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [15]:
model.add(layers.Dense(11))

In [16]:
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optim = keras.optimizers.Adam(lr=0.001)
metrics = ["accuracy"]

model.compile(optimizer=optim, loss=loss, metrics=metrics)

In [17]:
preprocess_input = tf.keras.applications.vgg16.preprocess_input

In [18]:
train_gen = keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_input)
valid_gen = keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_input)
test_gen = keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_input)

train_batches = train_gen.flow_from_directory(
    'D:/original_image/train',
    target_size = (240, 240),
    class_mode = 'sparse',
    batch_size = 2,
    shuffle=True,
    color_mode="rgb",
    classes=names
)

val_batches = valid_gen.flow_from_directory(
    'D:/original_image/val',
    target_size = (240, 240),
    class_mode = 'sparse',
    batch_size = 2,
    shuffle=False,
    color_mode="rgb",
    classes=names
)

test_batches = valid_gen.flow_from_directory(
    'D:/original_image/test',
    target_size = (240, 240),
    class_mode = 'sparse',
    batch_size = 2,
    shuffle=False,
    color_mode="rgb",
    classes=names
)

Found 6600 images belonging to 11 classes.
Found 2750 images belonging to 11 classes.
Found 1650 images belonging to 11 classes.


In [ ]:
# folder = "D:/original_image/test/moonlight"
# file_list = [x for x in os.listdir(folder)]

In [ ]:
# file_list

In [ ]:
# train_list=[]
# for i in tqdm(range(len(file_list))):
#     try:        
#         image = Image.open("D:/original_image/test/moonlight/" + file_list[i])
#         image.save("D:/error_image/" + str(i) + '.jpg')
#         train_list.append(image)
#     except:
#         print('error!!',file_list[i])

In [56]:
epochs = 15

early_stopping = keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=5
)

history = model.fit(train_batches, validation_data=val_batches,
         callbacks=[early_stopping],
         epochs=epochs)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 3300 steps, validate for 1375 steps
Epoch 1/15
3300/3300 [==============================] - 120s 36ms/step - loss: 0.0402 - accuracy: 0.9897 - val_loss: 0.1504 - val_accuracy: 0.9753
Epoch 2/15
3300/3300 [==============================] - 100s 30ms/step - loss: 0.0048 - accuracy: 0.9985 - val_loss: 0.0549 - val_accuracy: 0.9956
Epoch 3/15
3300/3300 [==============================] - 101s 31ms/step - loss: 0.0046 - accuracy: 0.9989 - val_loss: 0.0231 - val_accuracy: 0.9975
Epoch 4/15
3300/3300 [==============================] - 101s 31ms/step - loss: 0.0112 - accuracy: 0.9979 - val_loss: 0.0144 - val_accuracy: 0.9967
Epoch 5/15
3300/3300 [==============================] - 100s 30ms/step - loss: 1.2438e-05 - accuracy: 1.0000 - val_loss: 0.0204 - val_accuracy: 0.9964
Epoch 6/15
3300/3300 [==============================] - 101s 31ms/step - loss: 7.3427e-07 - accuracy: 1.0000 - val_loss: 0.0196 - val_accuracy: 0.9960
Epoch 7/15
330

In [57]:
loss, acc = model.evaluate(test_batches)
acc

  ...
    to  
  ['...']
825/825 [==============================] - 20s 24ms/step - loss: 1.5248e-04 - accuracy: 1.0000


1.0

In [58]:
model.save('model')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: model\assets


In [19]:
new_model = load_model('model')

In [45]:
# test_datagen = ImageDataGenerator(rescale=1./255)
# test_generator = test_datagen.flow_from_directory('D:/final_test/',target_size=(240,240),
#                                                         batch_size = 2,
#                                                         shuffle=False,
#                                                         color_mode="rgb",
#                                                         classes=names)
이거는 테스트 참고용

Found 33 images belonging to 11 classes.


In [20]:
new_model.evaluate(test_batches)

825/825 [==============================] - 21s 25ms/step - loss: 1.5248e-04 - accuracy: 1.0000


[0.00015248138950447944, 1.0]

In [34]:
datagen = ImageDataGenerator(
        rotation_range=2,
        width_shift_range=0.05,
        height_shift_range=0.05,
        shear_range=0.05,
        zoom_range = 0.1,
        horizontal_flip=False,
        fill_mode="nearest")

folder = "D:/final_test"

file_list = [x for x in os.listdir(folder)]

sub_folder = "D:/final_test/" + "63buliding"
sub_file_list = [x for x in os.listdir(sub_folder)]

for i in tqdm(range(len(file_list))):
    sub_folder = "D:/final_test/" + file_list[i]
    sub_file_list = [x for x in os.listdir(sub_folder)]
    for j in range(len(sub_file_list)):
        image = load_img("D:/final_test/" + file_list[i] +"/" + sub_file_list[j])  # PIL 이미지
        x = img_to_array(image)  # (3, 300, 300) 크기의 NumPy 배열
        x = x.reshape((1,) + x.shape)  # (1, 3, 300, 300) 크기의 NumPy 배열

#     아래 .flow() 함수는 임의 변환된 이미지를 배치 단위로 생성해서
#     지정된 `preview/` 폴더에 저장합니다.
        k = 0
        for batch in datagen.flow(x, batch_size=5,
                                  save_to_dir="D:/final_test", save_prefix="super_" + file_list[i], save_format="jpg"):
            k += 1
            time.sleep(0.01)
            if k > 50:
                break  # 이미지 20장을 생성하고 마칩니다

100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:55<00:00,  5.09s/it]


In [35]:
import glob

In [36]:
X = []
folder = "D:/final_test"
file_list = [x for x in os.listdir(folder)]

for i in range(len(file_list)):
    image = load_img("D:/final_test/" + file_list[i])
    data = np.asarray(image)
    X.append(data)

X = np.array(X)


In [37]:
X.shape


(1672, 240, 240, 3)

In [38]:
file_list

['super_63buliding_0_1046.jpg',
 'super_63buliding_0_1130.jpg',
 'super_63buliding_0_1227.jpg',
 'super_63buliding_0_1353.jpg',
 'super_63buliding_0_1494.jpg',
 'super_63buliding_0_1785.jpg',
 'super_63buliding_0_1791.jpg',
 'super_63buliding_0_1797.jpg',
 'super_63buliding_0_1950.jpg',
 'super_63buliding_0_1953.jpg',
 'super_63buliding_0_2070.jpg',
 'super_63buliding_0_2239.jpg',
 'super_63buliding_0_224.jpg',
 'super_63buliding_0_2328.jpg',
 'super_63buliding_0_2353.jpg',
 'super_63buliding_0_2401.jpg',
 'super_63buliding_0_2434.jpg',
 'super_63buliding_0_248.jpg',
 'super_63buliding_0_2551.jpg',
 'super_63buliding_0_2591.jpg',
 'super_63buliding_0_2696.jpg',
 'super_63buliding_0_27.jpg',
 'super_63buliding_0_2736.jpg',
 'super_63buliding_0_275.jpg',
 'super_63buliding_0_2893.jpg',
 'super_63buliding_0_29.jpg',
 'super_63buliding_0_2920.jpg',
 'super_63buliding_0_294.jpg',
 'super_63buliding_0_2978.jpg',
 'super_63buliding_0_2998.jpg',
 'super_63buliding_0_3067.jpg',
 'super_63bulidi

In [39]:
real_model = load_model('model')
prediction = real_model.predict(test_batches)


In [40]:
real_model.predict(test_batches)

array([[15.423, -28.910, -17.631, ..., -36.727, -21.397, -22.870],
       [-2.734, -18.518, -16.004, ..., -33.582, -38.551, -29.221],
       [30.853, -29.221, -20.837, ..., -40.041, -27.142, -27.955],
       ...,
       [-44.389, -21.388, -10.239, ..., -9.495, -28.800, 17.210],
       [-39.460, -19.889, -9.330, ..., -8.614, -28.517, 14.578],
       [-35.583, -17.458, -9.226, ..., -5.944, -27.512, 15.522]],
      dtype=float32)

In [41]:
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
cnt = 0

In [71]:
final_names = ["63building", "castle","general","indep_door","judgement_castle","lotte_tower","namsan","tapgol_park","seoulstation",\
        "coex","moonlight"]


# 이 비교는 그냥 파일들이 있으면 해당 파일과 비교. 카테고리와 함께 비교해서 진행하는 것은 _4 파일.
for i in prediction:
    pre_ans = i.argmax()  # 예측 레이블
    pre_ans_str = ''
    
    if pre_ans == 0: pre_ans_str = "63buliding"
    elif pre_ans == 1: pre_ans_str = "castle"
    elif pre_ans == 2: pre_ans_str = "coex"
    elif pre_ans == 3: pre_ans_str = "general"
    elif pre_ans == 4: pre_ans_str = "indep_door"
    elif pre_ans == 5: pre_ans_str = "judgement_castle"
    elif pre_ans == 6: pre_ans_str = "indep_door"
    elif pre_ans == 7: pre_ans_str = "moonlight"
    elif pre_ans == 8: pre_ans_str = "namsan"
    elif pre_ans == 9: pre_ans_str = "seoulstation"
    elif pre_ans == 10: pre_ans_str = "tapgol_park"
        
        
    else: pre_ans_str = "what the!!"
    if i[0] >= 0.8 : print("해당이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[1] >= 0.8:  print("해당이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[2] >= 0.8:  print("해당이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[3] >= 0.8 :  print("해당이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[4] >= 0.8:  print("해당이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[5] >= 0.8:  print("해당이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[6] >= 0.8 :  print("해당이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[7] >= 0.8:  print("해당이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[8] >= 0.8:  print("해당이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[9] >= 0.8:  print("해당이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[10] >= 0.8:  print("해당이미지는 "+pre_ans_str+"로 추정됩니다.")
        

    cnt += 1
    print(i.argmax()) #얘가 레이블 [1. 0. 0.] 이런식으로 되어 있는 것을 숫자로 바꿔주는 것.
#     즉 얘랑, 나중에 카테고리 데이터 불러와서 카테고리랑 비교를 해서 같으면 맞는거고, 아니면 틀린거로 취급하면 된다.
#     이걸 한 것은 _4.py에.

해당이미지는 63buliding로 추정됩니다.
0
해당이미지는 63buliding로 추정됩니다.
0
해당이미지는 63buliding로 추정됩니다.
0
해당이미지는 castle로 추정됩니다.
1
해당이미지는 castle로 추정됩니다.
1
해당이미지는 castle로 추정됩니다.
1
2
2
7
해당이미지는 general로 추정됩니다.
3
해당이미지는 general로 추정됩니다.
3
해당이미지는 general로 추정됩니다.
3
해당이미지는 indep_door로 추정됩니다.
4
해당이미지는 indep_door로 추정됩니다.
4
해당이미지는 indep_door로 추정됩니다.
4
해당이미지는 63buliding로 추정됩니다.
0
해당이미지는 judgement_castle로 추정됩니다.
5
해당이미지는 judgement_castle로 추정됩니다.
5
해당이미지는 indep_door로 추정됩니다.
6
해당이미지는 indep_door로 추정됩니다.
6
해당이미지는 indep_door로 추정됩니다.
6
해당이미지는 moonlight로 추정됩니다.
7
해당이미지는 moonlight로 추정됩니다.
7
해당이미지는 moonlight로 추정됩니다.
해당이미지는 moonlight로 추정됩니다.
7
해당이미지는 castle로 추정됩니다.
1
해당이미지는 castle로 추정됩니다.
1
해당이미지는 castle로 추정됩니다.
1
해당이미지는 seoulstation로 추정됩니다.
9
9
해당이미지는 seoulstation로 추정됩니다.
9
1
6
7
